# **Análisis Exploratorio de los Datos (EDA)**

En esta sección se tiene como objetivo realizar un análisis exploratorio del conjunto de datos. El propósito es comprender mejor las variables disponibles, identificar patrones generales, detectar valores atípicos y preparar el terreno para análisis posteriores.

Trabajaremos paso a paso a través de la carga, inspección, limpieza y visualización de los datos, utilizando Python y bibliotecas como `pandas`, `matplotlib`, y `seaborn`.


## 📦 **Importación de Librerías**

En esta sección cargamos las bibliotecas necesarias para la manipulación y visualización de datos.


In [1]:
# Importamos las librerías básicas necesarias para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import neighbors
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
# Establecemos el estilo de los gráficos
sns.set(style="whitegrid", palette="pastel")


## **📂 Cargar la base de datos**

Aquí leemos el archivo CSV que contiene los datos. Es importante verificar que se cargue correctamente y revisar las primeras filas para asegurarnos de que la estructura sea la esperada.


In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
predict_students_dropout_and_academic_success = fetch_ucirepo(id=697)

# data (as pandas dataframes)
X = predict_students_dropout_and_academic_success.data.features
y = predict_students_dropout_and_academic_success.data.targets

# metadata
print(predict_students_dropout_and_academic_success.metadata)

# variable information
print(predict_students_dropout_and_academic_success.variables)

{'uci_id': 697, 'name': "Predict Students' Dropout and Academic Success", 'repository_url': 'https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success', 'data_url': 'https://archive.ics.uci.edu/static/public/697/data.csv', 'abstract': "A dataset created from a higher education institution (acquired from several disjoint databases) related to students enrolled in different undergraduate degrees, such as agronomy, design, education, nursing, journalism, management, social service, and technologies.\nThe dataset includes information known at the time of student enrollment (academic path, demographics, and social-economic factors) and the students' academic performance at the end of the first and second semesters. \nThe data is used to build classification models to predict students' dropout and academic sucess. The problem is formulated as a three category classification task, in which there is a strong imbalance towards one of the classes.", 'area': 'Social Sc